# QIIME2: Terminal Run (Main Process)
---
Terminal (Shell command line code) for QIIME2

This pipeline consists of 2 files (2 Processes)
1. [Main Process](Terminal_Run_Main_Process.ipynb)
2. [Database Process](Terminal_Run_DB_Process.ipynb)
3. [Common (Final) Process](Terminal_Run_Common_Process.ipynb)

**To reduce running time the [all processes](#) should be run parallelly**

Documentation of QIIME2 - https://docs.qiime2.org/2019.7/tutorials/overview
Official website - https://qiime2.org

## Step of 16S
<img src="step.png"/>

# 0. Set Variable
- The variable should be set in [all processes](#)
- To reduce running time the [all processes](#) should be run parallelly

In [5]:
%env IN_MANIFEST="in.manifest.txt"
%env OTU_MANIFEST="otu.manifest.txt"

%env IN_FORMAT="PairedEndFastqManifestPhred33V2"

%env ADAPTER_FRW="ACACTGACGACATGGTTCTACA"
%env ADAPTER_REV="TACGGTAGCAGAGACTTGGTCT"


%env CPU_CORE=32
%env CPU_THREAD=0
%env CPU_JOB=-1

env: IN_MANIFEST="in.manifest.txt"
env: IN_FORMAT="PairedEndFastqManifestPhred33V2"
env: IN_SEQ_DB="/mnt/d/qiime2/Silva_132_release/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna"
env: IN_TAXONOMY="/mnt/d/qiime2/Silva_132_release/SILVA_132_QIIME_release/taxonomy/16S_only/99/consensus_taxonomy_7_levels.txt"
env: IN_DB_PERC=0.99
env: ADAPTER_FRW="ACACTGACGACATGGTTCTACA"
env: ADAPTER_REV="TACGGTAGCAGAGACTTGGTCT"
env: PRIMER_FRW="CCTACGGGNGGCWGCAG"
env: PRIMER_REV="GACTACHVGGGTATCTAATCC"
env: CPU_CORE=32
env: CPU_THREAD=0
env: CPU_JOB=-1


# 1. Import Seq and Demultiplex

In [6]:
%%bash
qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path  $IN_MANIFEST \
--input-format $IN_FORMAT \
--output-path 00.paired-end-demux.qza

# 2. Trim Adapter

In [ ]:
%%bash
qiime cutadapt trim-paired \
--i-demultiplexed-sequences 00.paired-end-demux.qza \
--p-cores $CPU_CORE \
--p-adapter-f $ADAPTER_FRW \
--p-adapter-r $ADAPTER_REV \
--o-trimmed-sequences 01.paired-end-remove-adapter.qza

# 3. Join Pair (If paired ends)

In [ ]:
%%bash
qiime vsearch join-pairs \
--i-demultiplexed-seqs 01.paired-end-remove-adapter.qza \
--p-minmergelen 367 \
--o-joined-sequences 03.demux-joined.qza

# 4. Quality filtering

In [ ]:
%%bash
qiime quality-filter q-score-joined \
--i-demux 03.demux-joined.qza \
--p-min-quality 20 \
--o-filtered-sequences 04.1.demux-joined-filtered.qza \
--o-filter-stats 04.2.demux-joined-filter-stats.qza

# 5. Dereplication

In [ ]:
%%bash
qiime vsearch dereplicate-sequences \
--i-sequences 04.1.demux-joined-filtered.qza \
--o-dereplicated-table 05.1.table.qza \
--verbose \
--o-dereplicated-sequences 05.2.rep-seqs.qza

# 6 - 7, 10. The Database Process
Run them in [Database Process notebook](Terminal_Run_DB_Process.ipynb)

# 8 - 9, 11. The Common Process  <a name="8"/>

<!--
**Can be run in both notebook**, in order to achieve shorter process time,
- If the Step 10 of [database process](Terminal_Run_DB_Process.ipynb) is not **complete**, **this step should be run here**.
- If another step is run in this main process (e.g. visualization) and the [database process](Terminal_Run_DB_Process.ipynb) has already run successfully,<br/>**this step should be run in [database process](Terminal_Run_DB_Process.ipynb#8)**.
-->
Run them in [Common Final Process](Terminal_Run_Common_Process.ipynb)

---
# Visualization

## 1. Visualize summary of sequencing
## 2. Visualize trimmed of sequencing

In [ ]:
%%bash
qiime demux summarize --i-data 00.paired-end-demux.qza --o-visualization vs00.paired-end-demux
qiime demux summarize --i-data 01.paired-end-remove-adapter.qza --o-visualization vs01.paired-end-remove-adapter

## 3. Visualize the joined pair end

In [ ]:
%%bash
qiime demux summarize --i-data 03.demux-joined.qza --o-visualization vs03.demuxjoined

## 4. Visualize the joined flitered sequence and stat

In [ ]:
%%bash
qiime demux summarize --i-data 04.1.demux-joined-filtered.qza --o-visualization vs04.1.demux-joined-filtered
qiime metadata tabulate --m-input-file 04.2.demux-joined-filter-stats.qza --o-visualization vs04.2.demux-joined-filter-stats

## 5.1. summarize feature table
## 5.2. display tabulate seq

In [ ]:
%%bash
qiime feature-table summarize \
--i-table 05.1.table.qza \
--o-visualization vs05.1.table
--m-sample-metadata-file $OTU_MANIFEST  ## metadata file is optional parameter


qiime feature-table tabulate-seqs \
--i-data 05.2.rep-seqs.qza \
--o-visualization vs05.2.rep-seqs